In [1]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
from google.cloud import vision
from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

# 다음 웹툰 썸네일크롤링
 - 작가 닉네임, 제목, 장르, 연령, 간략줄거리 -> 데이터프레임

In [201]:
def crawling_daum_webtoon_thumbnail():
    webtoon_info = []
    total_df_info = []
    total_img_count = 0
    rejected_data = []

    domain_url = 'http://webtoon.daum.net'
    day_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

    for each_day in day_list:
        json_url = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/{}?timeStamp=1588347908500'.format(each_day)
        json_data = requests.get(json_url).json()


        for each_data in json_data['data']:
            nickname = each_data['nickname']
            title = '&'.join(each_data['title'].split('/'))
            genre = ','.join([ele['name'] for ele in each_data['cartoon']['genres']])
            introduction = each_data['introduction']
            age = each_data['ageGrade']

            thumbnail_json_url = 'http://webtoon.daum.net/data/pc/webtoon/view/{}?timeStamp=1588399433358'.format(nickname)
            thumbnail_json_data = requests.get(thumbnail_json_url).json()

            webtoon_info.append({'nickname': nickname, 'title': title, 'genre': genre, 'content': introduction, 'age': age})

            if thumbnail_json_data['result']['status'] == '403':
                rejected_data.append({'title': title, 'message': thumbnail_json_data['result']['message']})
                continue

            if not os.path.exists('./imgs/{}'.format(title)):
                os.mkdir('./imgs/{}'.format(title))

            for each_data in thumbnail_json_data['data']['webtoon']['webtoonEpisodes']:
                urllib.request.urlretrieve(each_data['thumbnailImage']['url'], './imgs/{}/{}.png'.format(title, title + '_' + '&'.join(each_data['title'].split('/'))))
                urllib.request.urlretrieve(each_data['thumbnailImage']['url'], './total_img/{}.png'.format(title + '_' + '&'.join(each_data['title'].split('/'))))
                total_df_info.append({'nickname': nickname, 'title': title + '_' + '&'.join(each_data['title'].split('/')), 'genre': genre, 'content': introduction, 'age': age})
                total_img_count += 1


    df = pd.DataFrame(webtoon_info, columns=['nickname', 'title', 'genre', 'content', 'age'])
    total_df = pd.DataFrame(total_df_info, columns=['nickname', 'title', 'genre', 'content', 'age'])
    df.to_csv('webtoon_info.csv')
    total_df.to_csv('total_img_info.csv')


In [225]:
webtoon_info = []
total_df_info = []
total_img_count = 0
rejected_data = []

domain_url = 'http://webtoon.daum.net'
day_list = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

for each_day in day_list:
    json_url = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/{}?timeStamp=1588347908500'.format(each_day)
    json_data = requests.get(json_url).json()


    for each_data in json_data['data']:
        nickname = each_data['nickname']
        title = '&'.join(each_data['title'].split('/'))
        genre = ','.join([ele['name'] for ele in each_data['cartoon']['genres']])
        introduction = each_data['introduction']
        age = each_data['ageGrade']

        thumbnail_json_url = 'http://webtoon.daum.net/data/pc/webtoon/view/{}?timeStamp=1588399433358'.format(nickname)
        thumbnail_json_data = requests.get(thumbnail_json_url).json()
        
        webtoon_info.append({'nickname': nickname, 'title': title, 'genre': genre, 'content': introduction, 'age': age})
        
        if thumbnail_json_data['result']['status'] == '403':
            rejected_data.append({'title': title, 'message': thumbnail_json_data['result']['message']})
            continue

        if not os.path.exists('./imgs/{}'.format(title)):
            os.mkdir('./imgs/{}'.format(title))

        for each_data in thumbnail_json_data['data']['webtoon']['webtoonEpisodes']:
            urllib.request.urlretrieve(each_data['thumbnailImage']['url'], './imgs/{}/{}.png'.format(title, title + '_' + '&'.join(each_data['title'].split('/'))))
            urllib.request.urlretrieve(each_data['thumbnailImage']['url'], './total_img/{}.png'.format(title + '_' + '&'.join(each_data['title'].split('/'))))
            total_df_info.append({'nickname': nickname, 'title': title + '_' + '&'.join(each_data['title'].split('/')), 'genre': genre, 'content': introduction, 'age': age})
            total_img_count += 1
    
    
df = pd.DataFrame(webtoon_info, columns=['nickname', 'title', 'genre', 'content', 'age'])
total_df = pd.DataFrame(total_df_info, columns=['nickname', 'title', 'genre', 'content', 'age'])
df.to_csv('webtoon_info.csv')
total_df.to_csv('total_img_info.csv')


In [226]:
total_img_count

10950

In [234]:
list(df['title'])

['현세이의 보이는 라디오',
 '강호표사',
 '사귄 건 아닌데',
 '하나밖에 안 보여!',
 '메모리스트',
 '쓰레기 머학생',
 '어름치',
 '식귀',
 '아싸가 알아버렸다',
 '찬란한 액션 유치원',
 '정상회담 : 스틸레인3',
 '흑우',
 '타원을 그리는 법',
 '교수님을 빚는 중',
 '조류공포증',
 '이대로 멈출 순 없다',
 '망자의 서',
 '그라티아',
 '굿바이 사돈!',
 '리세터',
 '노동본색',
 '집행자여',
 'NR 뉴월드',
 '일단 질러! 질렐루야',
 '트레이스',
 '비밀 사이',
 '아싸클럽',
 '너를 만나다',
 '뱀파이어 신드롬',
 '화폐개혁',
 '이태원 클라쓰',
 '그녀의 앵무새',
 '닥터 파인의 하루',
 '단두(斷頭)',
 '해오와 사라',
 '신성한, 이혼',
 '멸종인간',
 '우리집이거든요!',
 '학교대표',
 '거울 죽이기',
 '헬로 마이 안드로이드',
 '겟 레디 위드 미',
 '아티스트',
 '계약우정',
 '아비무쌍',
 '지옥사원',
 '왕 그리고 황제',
 '인간을 먹는 성',
 '나는 엄마다',
 '아버지의 복수는 끝이 없어라',
 '노력의 卍(만)',
 '죽어도! 치우',
 '용의주도 황비',
 '금수친구들',
 '설공찬전',
 'Breaker.Drawing',
 '말하지 말까',
 '해치지않아',
 '하나하나둘',
 '콜로세움',
 '4학년',
 '키스해주세요',
 '토이 콤플렉스',
 '대나무숲에서 알립니다',
 '돈나무',
 '우리가 사랑하는 방법',
 '후크',
 '헤븐즈 소울',
 '동쪽으로',
 '쌍갑포차',
 '이게뭐야',
 '수의 계절',
 '블랙 베히모스',
 '창백한 말',
 '살어리랏다',
 '고교호구왕',
 '스타의 학교생활',
 '냥냐라냥냥',
 '메모리스트',
 '옆집남자',
 '신의 태궁',
 '재벌과의 인터뷰',
 '이담',
 '19',
 '샬롯에게는 다섯 명의 제자가 있다',
 '특별직',
 '개같은 세상',
 '저기요 손님',
 '블랙

In [235]:
total_df

,nickname,title,genre,content,age
0,sayradio,현세이의 보이는 라디오_6화,"드라마,일상",스트리머 현세이의 일상 소통 이야기,0
1,sayradio,현세이의 보이는 라디오_5화,"드라마,일상",스트리머 현세이의 일상 소통 이야기,0
2,sayradio,현세이의 보이는 라디오_4화,"드라마,일상",스트리머 현세이의 일상 소통 이야기,0
3,sayradio,현세이의 보이는 라디오_3화,"드라마,일상",스트리머 현세이의 일상 소통 이야기,0
4,sayradio,현세이의 보이는 라디오_2화,"드라마,일상",스트리머 현세이의 일상 소통 이야기,0
...,...,...,...,...,...
10945,parody,채널뽁스_편성 2. 뽁 아이덴티티 (1),"코믹,지식",상상 그 이상의 요절복통 코미디 멀티플렉스 <채널뽁스>,0
10946,parody,채널뽁스_편성 1. 잉여스텔라 (3),"코믹,지식",상상 그 이상의 요절복통 코미디 멀티플렉스 <채널뽁스>,0
10947,parody,채널뽁스_편성 1. 잉여스텔라 (2),"코믹,지식",상상 그 이상의 요절복통 코미디 멀티플렉스 <채널뽁스>,0
10948,parody,채널뽁스_편성 1. 잉여스텔라 (1),"코믹,지식",상상 그 이상의 요절복통 코미디 멀티플렉스 <채널뽁스>,0
